In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from random import random

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
from models import ModelManager, ModelType
from utils import MnistData

In [ ]:
data = MnistData()

## Training

In [ ]:
generator = ModelManager.get_untrained(ModelType.Generator)
discriminator = ModelManager.get_untrained(ModelType.WassersteinCritic)

In [ ]:
batch = 100
# batch_of_real_data, _ = data.get_training_batch(batch)
# batch_of_noise = torch.rand(batch, 100)

In [ ]:
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=2e-4, weight_decay=0.1)
generator_optimizer = optim.Adam(generator.parameters(), lr=2e-4, weight_decay=0.03)


for epoch in range(600 * 200):
    # sampling
    batch_of_real_data, _ = data.get_training_batch(batch)
    batch_of_noise = torch.rand(batch, 100)
    
    # zero grad
    generator_optimizer.zero_grad()
    discriminator_optimizer.zero_grad()

    # predictions
    batch_of_generated = generator(batch_of_noise)
    batch_of_generated_discrimination = discriminator(batch_of_generated)
    batch_of_real_discrimination = discriminator(batch_of_real_data)

    # computing loss
    discriminator_loss = batch_of_generated_discrimination.mean() - batch_of_real_discrimination.mean()
    # discriminator_loss /= 2
    discriminator_loss.backward()

    discriminator_optimizer.step()

    batch_of_generated = generator(batch_of_noise)
    batch_of_generated_discrimination = discriminator(batch_of_generated)

    generator_loss = - batch_of_generated_discrimination.mean() - 0.3 * batch_of_generated.std(dim=0).sum()
    discriminator_optimizer.zero_grad()
    generator_optimizer.zero_grad()
    generator_loss.backward()

    generator_optimizer.step()
    # log
    if (epoch + 1) % 100 == 0:
        print(f'---{epoch + 1}:D:{discriminator_loss.item()}:G:{generator_loss.item()}---')
        MnistData.display(batch_of_generated[0], scale=True)

## Check

In [ ]:
examples = generator(torch.rand(10, 100))

for i in range(10):
    MnistData.display(examples[i], scale=True)